In [ ]:
# import os
# os.getcwd()
# os.chdir("../../..")
# os.getcwd()

We can use Hypergraph features for various predictive tasks:

In [ ]:
import convokit
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
corpus = convokit.Corpus(filename=convokit.download("reddit-corpus"))

In [ ]:
remake_cache = True
if remake_cache:
    hc = convokit.HyperConvo()
    hc.fit_transform(corpus)
    feats = corpus.get("meta")["hyperconvo"]

    with open("feats.p", "wb") as f:
        pickle.dump(feats, f)
else:
    with open("feats.p", "rb") as f:
        feats = pickle.load(f)

In [ ]:
list(feats.keys())[:3]

In [ ]:
from collections import defaultdict

In [ ]:
threads = corpus.utterance_threads()

In [ ]:
# Use only the first 10 comments in each thread
thread_pfxs = corpus.utterance_threads(prefix_len=10)

In [ ]:
thread_roots_by_self_post = defaultdict(list)
for root, thread in threads.items():
    thread_roots_by_self_post[thread[root].user.info["self-post-id"]].append(root)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import random
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(2019)

for task in ["comment-growth", "commenter-growth", "post-deleted", "user-deleted"]:
    print("task {}".format(task))
    pos, neg = [], []
    for roots in thread_roots_by_self_post.values():
        has_pos, has_neg = [], []
        for root in roots:
            if task == "comment-growth":
                if len(threads[root]) >= 15:
                    has_pos.append(root)
                if len(threads[root]) == 10:
                    has_neg.append(root)
            elif task == "commenter-growth":
                if len(threads[root]) >= 20:
                    if len(set(c.user.name for c in threads[root].values())) >= \
                        len(set(c.user.name for c in thread_pfxs[root].values())) * 2:
                            has_pos.append(root)
                    else:
                        has_neg.append(root)
            elif task == "post-deleted":
                if len(threads[root]) >= 10:
                    if threads[root][root].user.info["post-deleted"]:
                        has_pos.append(root)
                    else:
                        has_neg.append(root)
            elif task == "user-deleted":
                if len(threads[root]) >= 10:
                    if threads[root][root].user.info["user-deleted"]:
                        has_pos.append(root)
                    else:
                        has_neg.append(root)
            else:
                print("unrecognized task name")

        if has_pos and has_neg:
            pos.append(random.choice(has_pos))
            neg.append(random.choice(has_neg))

    print("- {} positive, {} negative pts".format(len(pos), len(neg)))

    # make data from pos and neg
    X = []
    X_volume, X_reply, X_bow = [], [], []
    threads_text = []
    for root in pos + neg:
        # get ordered set of feature values
        v = [feats[root][k] for k in sorted(feats[root].keys()) if "TRIADS" not in k]
        # data cleaning
        v = [t if (not np.isnan(t) and np.isfinite(t)) else 0 for t in v]
        X.append(v)
        
        # volume baseline - get num participants in thread with at least length of 10
        X_volume.append([len(set(c.user.name for c in thread_pfxs[root].values()))])   
        # reply tree baseline
        X_reply.append([feats[root][k] if (not np.isnan(feats[root][k]) and np.isfinite(feats[root][k])) else 0 for k in sorted(feats[root].keys()) 
                        if "c->c" in k])
        # BOW baseline text
        thread_text = " ".join([u.text for u in thread_pfxs[root].values()
                                if not (task == "post-deleted" and u.id == root)])  
        # don't consider root post for post-deleted task, since we could just look for the string "[deleted]"
        threads_text.append(thread_text)
        
    ys = [1]*len(pos) + [0]*len(neg)

    X, ys = np.array(X), np.array(ys)

    for X_tmp, name in [(X, "hyperconvo"), (X_volume, "volume"), (X_reply, "reply tree"), (None, "BOW")]:
        if name == "BOW":
            text_train, text_test, y_train, y_test = train_test_split(threads_text, ys, test_size=0.1, random_state=42)
            cv = CountVectorizer(min_df=0.05, max_df=0.8)
            X_train = cv.fit_transform(text_train)
            X_test = cv.transform(text_test)
        else:
            X_train, X_test, y_train, y_test = train_test_split(X_tmp, ys, test_size=0.1, random_state=42)
        
        clf = LogisticRegression(solver="liblinear")
        clf.fit(X_train, y_train)

        train_acc = clf.score(X_train, y_train)
        test_acc = clf.score(X_test, y_test)
        print("- {}: {:.4f} train, {:.4f} test".format(name, train_acc, test_acc))